In [ ]:
from bs4 import BeautifulSoup
from time import sleep
import requests
import pandas as pd
from fake_useragent import UserAgent

url = "https://www.labirint.ru/genres/3000/"

response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
# for key, value in response.request.headers.items():
# print(key+": "+value)

# print(response)
soup = BeautifulSoup(response.text, 'html.parser')

starting_page = 'https://www.labirint.ru'

Загрузим необходимые библиотеки. Библиотеку fake_useragent будем использовать, чтобы роботы не блокировали нашу кражу информации о книжках.
Парсить будем сайт лабиринт. Пройдемся по всем жанрам и выберем там самые новые книги

In [ ]:
types = soup.find_all("a", rel="nofollow")
list_of_genres = []
list_of_genres_without_href = []
name_of_genres = []
for tipe in types:
    if tipe.text != '' and tipe.get('href')[:5] == "/genr" and tipe.text not in name_of_genres:
        sublist = [tipe.text, tipe.get('href')]
        name_of_genres.append(tipe.text)  # name of table
        list_of_genres.append(sublist)
        list_of_genres_without_href.append(tipe.text)

Изучив HTML разметку страницы приходим к очевидному выводу, что ссылки на странички с жанрами лежат в тегах < a > с классом "nofollow"
Нам нужно найти все жанры которые предложены сайтом и найти ссылки, которые переводят нас на эти странички

In [ ]:
list_of_book_hrefs = []
list_of_books_names = []
list_of_books_descriptions = []
list_of_books_images = []
list_of_books_genres = []

about_books = []
for i in range(len(list_of_genres)):
    new_url = starting_page + list_of_genres[i][1]
    # print(new_url)
    new_response = requests.get(new_url, headers={'User-Agent': UserAgent().chrome})
    genre = BeautifulSoup(new_response.text, 'html.parser')
    books = genre.find_all("a", class_="cover cover-tooltip")
    for book in books:
        list_of_books_genres.append(list_of_genres[i][0])
        about_books.append([list_of_genres[i][0], book.get('title')])
        list_of_book_hrefs.append(book.get('href'))
        list_of_books_names.append(book.get('title'))
        sleep(1)
        list_of_books_images.append(book.parent.parent.find("img").get("data-src"))
        book_url = starting_page + book.get('href')
        sleep(1)
        print(book_url)
        book_response = requests.get(book_url, headers={'User-Agent': UserAgent().chrome})
        # print(book_response)
        detail = BeautifulSoup(book_response.text, 'html.parser')
        sleep(1)
        detail_ = detail.find(id="fullannotation")
        if detail_ is not None:
            list_of_books_descriptions.append(detail_.p)
        else:
            list_of_books_descriptions.append(None)


Перейдя на сайт с жанрами, для каждой ссылки ищем необходимое поле с новыми книгами в этой области. Так как книг, относящихся к конкретному жанру очень много и не все они новы, то парсить имеет смысл только те жанры, в которых есть обновления недавние. Мы находим названия книг, описания, ссылку на изображение и ссылку на книгу в интернет магазине

In [ ]:
boooks_dict = {"list_of_book_names": list_of_books_names, 'list_of_books_descriptions': list_of_books_descriptions,
               "list_of_books_images": list_of_books_images, "list_of_book_hrefs": list_of_book_hrefs,
               "list_of_books_genres": list_of_books_genres}

BookData = pd.DataFrame(boooks_dict, columns=boooks_dict.keys())
BookData.to_csv('BookData.csv')

После того, как все запарсили, создадим табличку, в которой будем хранить необходимые данные для отправки пользователю. Сохраним ее.

In [ ]:
import sqlite3

conn = sqlite3.connect('books.db')

cursor = conn.cursor()

try:
    user = "CREATE TABLE \"books\" (\"ID\" INTEGER UNIQUE, \"user_id\" INTEGER, \"book\" TEXT, \"ReadUnread\" TEXT, \"IMPRESSION\", PRIMARY KEY (\"ID\"))"
    cursor.execute(user)
except:
    pass

Так как бот не только предлагает читателю книги, но еще и интересуется его мнением по поводу прочитанного, нам необходимо создать базу данных, а также табличку внутри нее, чтобы сохранять то, что пишет пользователь. В итоговой версии проекта столбец ReadUnread не используется, но в следующих версиях бота он будет необходим.

## Приступим к написанию бота

In [ ]:
# import config - импортируем файл с токеном
import sqlite3
import telebot
from telebot import types
import pandas as pd
import random as rd

bot = telebot.TeleBot(config.token)


@bot.message_handler(commands=['start'])
def send_keyboard(message, text="Привет, хочешь поговорить на книжные темы?)"):
    keyboard_start = types.ReplyKeyboardMarkup(row_width=3)
    itembtn1 = types.KeyboardButton('Рассказать о книге')
    itembtn3 = types.KeyboardButton('Посмотреть запись о книге')
    itembtn4 = types.KeyboardButton("Найти новую книгу")
    itembtn5 = types.KeyboardButton('Пока все!')
    keyboard_start.add(itembtn1, itembtn4)
    keyboard_start.add(itembtn3)
    keyboard_start.add(itembtn5)

    msg = bot.send_message(message.from_user.id,
                           text=text, reply_markup=keyboard_start)
    bot.register_next_step_handler(msg, callback_worker)

Первое чтто нужно сделать -- это создать клавиатуру для бота

Будем добавлять в бота функции

In [ ]:
def add_impression(msg):
    with sqlite3.connect('books.db') as con:
        user_message = msg.text
        name_and_author = ''
        impression = ''
        key = 0
        for i in range(len(user_message)):
            if user_message[i] == ")":
                key = i
            if key == 0 and user_message[i] != "(" and user_message[i] != ')':
                name_and_author += user_message[i]
            elif user_message[i] != ")" and user_message[i] != '(':
                impression += user_message[i]
        cursor = con.cursor()
        cursor.execute('INSERT INTO books (user_id, book, IMPRESSION, ReadUnread) VALUES (?, ?, ?, ?)',
                       (msg.from_user.id, name_and_author, impression, "READ"))
        con.commit()
    bot.send_message(msg.chat.id, 'Очень интересный рассказ! Буду знать больше про эту книгу:)')
    send_keyboard(msg, "Что-то еще интересное про книжки?")

Первая функция работает с базой данных и спрашивает у пользователя про впечатление о прочитанной книге. Пользователь вводит название книги и впечатление в указанном формате. 

In [ ]:
def send_keyboard_3(message, text="Выбери жанр)"):
    bot.send_message(message.chat.id, 'Отлично)(')
    keyboard_start_3 = types.ReplyKeyboardMarkup(row_width=3)
    itembtn1 = types.KeyboardButton('Бизнес. Экономика')
    itembtn3 = types.KeyboardButton('Естественные науки')
    itembtn4 = types.KeyboardButton("Книги для родителей")
    itembtn5 = types.KeyboardButton('Кулинария')
    itembtn6 = types.KeyboardButton('Медицина и здоровье')
    itembtn7 = types.KeyboardButton('Психология')
    itembtn8 = types.KeyboardButton('Публицистика')
    itembtn9 = types.KeyboardButton('Главные книги отдела')
    keyboard_start_3.add(itembtn1, itembtn3, itembtn4)
    keyboard_start_3.add(itembtn5, itembtn6, itembtn7)
    keyboard_start_3.add(itembtn8, itembtn9)
    msg = bot.send_message(message.from_user.id,
                           text=text, reply_markup=keyboard_start_3)
    bot.register_next_step_handler(msg, find_new)

Будем использовать другую клавиатуру для того, чтобы упростить пользователю поиск книги по жанру

In [ ]:
def find_new(msg):
    user_message = msg.text
    pd.set_option('display.max_colwidth', 100000)
    BookData = pd.read_csv('BookData.csv')
    genre = BookData.loc[BookData['list_of_books_genres'] == user_message]
    Size = genre.shape[1]
    number = rd.randint(0, Size)
    srez = genre.iloc[[number], :]
    name = srez['list_of_book_names']
    description = str(srez['list_of_books_descriptions'].tolist()[0])
    image = srez['list_of_books_images'].tolist()[0]
    href = 'https://www.labirint.ru' + srez['list_of_book_hrefs']
    print(image)
    print(description)
    bot.send_photo(msg.chat.id, image)
    if description is None or description == 'nan':
        answer = name + '\n' + href
    else:
        description = description.replace('<p>', '')
        description = description.replace('<br', "")
        description = description.replace("</p>", '')
        description = description.replace('<br/', '')
        description = description.replace('/>', '')
        print(description)
        answer = name + '\n' + description + '\n' + href
    bot.send_message(msg.chat.id, answer)

    send_keyboard(msg, "Что-то еще интересное про книжки?")

Перейдем к функции, которая отвечает за поиск новой книги. В процессе парсинга мы столкнулись с проблемой: описание книги не обращается в обычный текст из-за тегов <br>, поэтому ручками уберем все эти теги. Также не для всех книг нашлось описание, поэтому некоторые книги будут представлены в формате название + изображение + ссылка

In [ ]:
def get_impression_string(impr):
    impr_str = []
    for val in list(enumerate(impr)):
        impr_str.append(str(val[1][2]) + '-' + str(val[1][4]) + '\n')
    return ''.join(impr_str)


def show_impressions(msg):
    print("We are here")
    with sqlite3.connect('books.db') as con:
        cursor = con.cursor()
        cursor.execute('SELECT * FROM books WHERE user_id=={}'.format(msg.from_user.id))
        tasks = get_impression_string(cursor.fetchall())
        bot.send_message(msg.chat.id, tasks)
        send_keyboard(msg, "Чем еще могу помочь?")

Обратимся к базе данных, чтобы вытащить все записи пользователя о книге с помощью второй функции, а с помощью первой функции выведем записи в аккуратном формате

In [ ]:
def callback_worker(call):
    if call.text == "Рассказать о книге":
        msg = bot.send_message(call.chat.id,
                               'Давай обсудим твои впечатления о книге! Напиши название книги и свое впечатление в следующем формате: \n (Название, автор)  Впечатления о книге')
        bot.register_next_step_handler(msg, add_impression)

    elif call.text == "Найти новую книгу":
        msg = bot.send_message(call.chat.id, 'Процесс выполняется, как ваше настроение?')
        bot.register_next_step_handler(msg, send_keyboard_3)

    elif call.text == "Посмотреть запись о книге":
        try:
            bot.register_next_step_handler(call, show_impressions)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Если хочешь, можем подобрать тебе книгу прямо сейчас!')
            send_keyboard(call, "Чем еще могу помочь?")

    elif call.text == "Пока все!":
        bot.send_message(call.chat.id, 'Хорошего дня! Когда захотите продолжнить нажмите на команду /start')

Создадим главную функцию для обработки запросов к боту

In [ ]:
if __name__ == '__main__':
    bot.infinity_polling()

### Заверщающий момент